# Refinitiv Data Library for Python
## Content - Pricing - Synchronous calls

This notebook demonstrates how to use the Pricing interface to retrieve data from the Refinitiv Data Platform.

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session('rdp')

## Get Price Snapshot
The following gets a snapshot from the non-streaming RDP Pricing API - note the **get_data()** call

In [3]:
# Define our RDP Snapshot Price object
response = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

response.data.df

,Instruments,BID,ASK
0,EUR=,1.2129,1.213
1,GBP=,1.4087,1.4088
2,JPY=,110.07,110.08
3,CAD=,1.2178,1.2182


## Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a **Deployed** session.   
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed - note the **get_stream()** call

In [4]:
# Define our Streaming Price object
streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
streaming.open()
# When you want the current price, get a snapshot
streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.2129,1.2133
1,GBP=,1.4086,1.409
2,JPY=,110.06,110.09
3,CAD=,1.2179,1.218


#### Call get_snapshot() again, prices should change (assuming instruments are being traded/quoted)

In [5]:
streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.2129,1.2133
1,GBP=,1.4086,1.409
2,JPY=,110.06,110.09
3,CAD=,1.2179,1.218


### Snapshot only - no streaming required
If you need to snap the price once and don't need further updates, you can make a non-streaming request

In [6]:
# If you want snap just the current prices, don't need updates
non_streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)
# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.2129,1.2133
1,GBP=,1.4087,1.4088
2,JPY=,110.07,110.08
3,CAD=,1.2179,1.218


The *non_streaming* instruments are closed off - **no need to close them**.   


### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [7]:
streaming['EUR=']['BID']

1.213

In [8]:
eur = streaming['CAD=']
eur['BID']

1.2179

#### Iterate on fields

In [9]:
print('JPY=')
for field_name, field_value in streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 110.07
	ASK : 110.08


#### Iterate on Streaming instruments and fields

In [10]:
for instrument in streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.213
	ASK : 1.2131
GBP=
	BID : 1.4087
	ASK : 1.4088
JPY=
	BID : 110.07
	ASK : 110.08
CAD=
	BID : 1.2179
	ASK : 1.218


### Close the Streaming Items when no longer required

In [11]:
streaming.close()

<StreamState.Closed: 1>

## Close the session

In [12]:
close_session()